In [16]:
import rpyc
from rpyc.utils.server import Server, ThreadedServer
from rpyc import async_
from threading import Thread, Lock, RLock
import time
import copy
from collections import deque
import itertools

mt = RLock()

class Myservice(rpyc.Service):
    def __init__(self):
        self.ls_child = ls_child
        self.child_conn = []
        self.bgthreads = []
        self.func = None
        self.async_objs = []
        self.l = deque(maxlen = 1000)
        self.id_list = deque(maxlen = 1000)
        self.shadow = None
        self.gran = 5
        self.lc = 0
        self.id = 21
        self.counter = 1
        self.finishes = {}
        self.pkt_details = {}
        self.is_shadow = False
        
        for ch in self.ls_child:
            c = rpyc.connect(ch[0], ch[1])
            self.child_conn.append(c)
            self.bgthreads.append(rpyc.BgServingThread(c))
        if(self.is_shadow):
            self.sent_packets = []
        else:
            self.shadow = shadow
            self.shadow_conn = rpyc.connect(self.shadow[0], self.shadow[1])
            self.shadow_bg = rpyc.BgServingThread(self.shadow_conn)   
            
    def on_finish(self):

        if(len(self.finishes) == (len(self.child_conn) + 1)):
            print("enter")
            print(self.lc)
            while(len(self.l) > self.lc):
                send = self.gran
                if (len(self.l) - self.lc < self.gran):
                    send = len(self.l) - self.lc
                self.pkt_details[str(self.counter)] = (self.lc, send)
                pkt = list(itertools.islice(self.l, self.lc, self.lc+send))
                self.func(self.counter, pkt, self.id, self.ack_func)
                for _ in range(send):
                    self.l.popleft()
                self.counter += 1
                print("on_finish waala {}".format(self.counter))
#                 self.lc += send
    
    def exposed_packet_ack(self, l):
        self.sent_packets.extend(l)
        print(l)
        
    def exposed_print_pkt_list(self):
        print(self.sent_packets)
    
    def ack_func(self, ID):
        print("Ack recieved for packet id {}".format(ID))
        vals = self.pkt_details[str(ID)]
        id_pkt = list(itertools.islice(self.id_list,vals[0],vals[0]+vals[1]))
        self.shadow_conn.root.packet_ack(id_pkt)
    
    
    def cback(self, pkt_id, value, ID, ack_func, fin = False):
        global mt
        mt.acquire()
        try:
            if(fin == True):
                print("fin called by {}".format(ID))
                self.finishes[str(ID)] = True
                self.on_finish()
            else:                
                idl = []
                for _ in range(len(value)):
                    idl.append(ID)
                self.id_list.extend(idl)
                self.l.extend(value)

                if (ID != self.id):
                    ack_func(pkt_id)

                while(len(self.l) - self.lc >= self.gran):
                    send = self.gran
                    self.pkt_details[str(self.counter)] = (self.lc, self.gran)
                    pkt = list(itertools.islice(self.l, self.lc, self.lc+send))
                    self.func(self.counter, pkt, self.id, self.ack_func)
                    for _ in range(send):
                        self.l.popleft()
                        self.id_list.popleft()
                    self.counter += 1
                    print("cback waala {}".format(self.counter))
#                     self.lc += self.gran
            
        except Exception as e:
            print(e)
        finally:
            mt.release()
        return
    
    def on_connect(self, conn):
        print("Connection established with "+ str(conn))
        
    def on_disconnect(self,conn):
        print("Have a nice day... goodbye")
        
    def exposed_printvals(self):
        print(self.l)
        print(self.id_list)
    
    def local_search(self):
        fn = self.cback
        for j in range(10):
            l = []
            for i in range(200 + 10*j,200 + 10*j + 10):
                l.append(i)


            time.sleep(1)
            print("done " + str(j))
            fn(j, l, self.id, self.ack_func)
            print("sent " + str(j))
        
        fn(None, None, self.id, None, True)   
            
    def exposed_do_a_search(self, func):
        global mt
        self.func = func
        for child in self.child_conn:
            asearch = async_(child.root.do_a_search)
            aobj = asearch(self.cback)
            self.async_objs.append(aobj)

        
        self.local_search()

In [17]:
ls_child = []
ls_child.append(("localhost", 18850, 11))
ls_child.append(("localhost", 18851, 12))

shadow = ["localhost", 17752]

In [ ]:
t = ThreadedServer(Myservice, port = 18852)
t.start()

Connection established with <rpyc.core.protocol.Connection 'conn24' object at 0x7ff8c6f66b00>
done 0
Ack recieved for packet id 1
cback waala 2
Ack recieved for packet id 2
cback waala 3
sent 0
Ack recieved for packet id 3
cback waala 4
Ack recieved for packet id 4
cback waala 5
done 1
Ack recieved for packet id 5
cback waala 6
Ack recieved for packet id 6
cback waala 7
sent 1
Ack recieved for packet id 7
cback waala 8
Ack recieved for packet id 8
cback waala 9
done 2
Ack recieved for packet id 9
cback waala 10
Ack recieved for packet id 10
cback waala 11
sent 2
Ack recieved for packet id 11
cback waala 12
Ack recieved for packet id 12
cback waala 13
done 3
Ack recieved for packet id 13
cback waala 14
Ack recieved for packet id 14
cback waala 15
sent 3
Ack recieved for packet id 15
cback waala 16
Ack recieved for packet id 16
cback waala 17
done 4
Ack recieved for packet id 17
cback waala 18
Ack recieved for packet id 18
cback waala 19
sent 4
Ack recieved for packet id 19
cback waala 2

In [7]:
d = {}
d['abc'] = 1
d['cde'] = 2

In [10]:
for key,value in d.items():
    print(key, value)

cde 2
abc 1


In [14]:
def f(kl):
    kl[0] = 1
    

In [22]:
l = [0,2,3]

In [23]:
f(l[0:1])

In [24]:
l

[0, 2, 3]

In [110]:
s = 3
if (s != 3):
    print(s)